<a href="https://colab.research.google.com/github/VictorSwankyTiger/DM2024-Lab1-Homework/blob/main/DataLab_Cup1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = '2024-datalab-cup1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F85283%2F9621689%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241005%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241005T180948Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0c4292348f830f7e44db9f381cea3476c4cbff5d4e577abf8cf7af2b58a58c0d8c9ee6b6a386d56191e8eaab821ecda96e896e91a8901c57dd4ad1e12286ad1a073a8d4ccb4b5f542017c6c4df9a7ca65938e28ffb603bcdb131f587958f124cec19e3618cf3b2f9ac5258e512f68f119dc1b03cbe587787827309711b18fb8b70f9d6a3e7d27025739d804e0cedf68eab68146339dda1f4312b75b2f5bcb671437d6d8191ba133bc74cd1f95664b37ccd2f23683318ea45879174f2c7f27959893f3e2c7f5b2fe449ceb910d716559f24bf272c621c6f6621b9fb77df1e217dbc856e382f0a5fd666a523f1a7a4d19df270bb04f50dd462ef3d507410c55245'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 98289203 bytes downloaded
Downloaded and uncompressed: 2024-datalab-cup1
Data source import complete.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2024-datalab-cup1/train.csv
/kaggle/input/2024-datalab-cup1/test.csv
/kaggle/input/2024-datalab-cup1/sample_submission.csv


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
%matplotlib inline

In [20]:
import pandas as pd

df = pd.read_csv('/kaggle/input/2024-datalab-cup1/train.csv')
#df = pd.read_csv('./dataset/sentiment/train.csv')
df = pd.DataFrame(df)
print(df.head(5))

   Id  Popularity                                       Page content
0   0          -1  <html><head><div class="article-info"> <span c...
1   1           1  <html><head><div class="article-info"><span cl...
2   2           1  <html><head><div class="article-info"><span cl...
3   3          -1  <html><head><div class="article-info"><span cl...
4   4          -1  <html><head><div class="article-info"><span cl...


In [10]:
import re
from bs4 import BeautifulSoup

def preprocessor(text):
    # remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # regex for matching emoticons, keep emoticons, ex: :), :-P, :-D
    r = '(?::|;|=|X)(?:-)?(?:\)|\(|D|P)'
    emoticons = re.findall(r, text)
    text = re.sub(r, '', text)

    # convert to lowercase and append all emoticons behind (with space in between)
    # replace('-','') removes nose of emoticons
    text = re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-','')
    return text

In [21]:
#print(df.loc[0,'Page content'])
df['Page content']=df['Page content'].apply(preprocessor)
print(df.head(5))


   Id  Popularity                                       Page content
0   0          -1   clara moskowitz for space com 2013 06 19 15 0...
1   1           1  by christina warren2013 03 28 17 40 55 utcgoog...
2   2           1  by sam laird2014 05 07 19 15 20 utcballin 2014...
3   3          -1  by sam laird2013 10 11 02 26 50 utccameraperso...
4   4          -1  by connor finnegan2014 04 17 03 31 43 utcnfl s...


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
stop = stopwords.words('english')

def tokenizer(text):
    porter = PorterStemmer()
    if isinstance(text, str):
        return [porter.stem(w) for w in re.split('\s+', text.strip()) \
                if w not in stop and re.match('[a-zA-Z]+', w)]
    elif isinstance(text, list):
        # Join list elements into a single string
        text_str = ' '.join(text)
        return [porter.stem(w) for w in re.split('\s+', text_str.strip()) \
                if w not in stop and re.match('[a-zA-Z]+', w)]
    else:
        # Handle other data types if necessary
        return []

df['Page content']=df['Page content'].apply(tokenizer)
print(df.head(5))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
